In [ ]:
import subprocess
from Bio.Blast import NCBIXML
import csv
from Bio import SeqIO

query_fasta = '/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb_16_msa_filt_proteins.fasta'
# Path to the output XML file
blast_output = "/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb_16_blast_output.xml"
# Path to BLAST database
database = "/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/blast_db/at_db_adj"
results_csv = "/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb_16_blast_output_adj.csv"
e_value_threshold = 0.005

# Construct the BLAST command as a list
blast_command = [
    '/home/adititm/ncbi-blast-2.15.0+/bin/blastp',
    '-query', query_fasta,
    '-db', database,
    '-evalue', str(e_value_threshold),
    '-outfmt', '5',  # XML format
    '-out', blast_output,
    '-num_threads', '2'
]

# Run the BLAST command
subprocess.run(blast_command, check=True)

# Parse query sequences into a dictionary
query_sequences = {record.id: str(record.seq) for record in SeqIO.parse(query_fasta, "fasta")}

# Open the BLAST XML output for parsing within a single continuous context
with open(blast_output) as result_handle:
    blast_records = NCBIXML.parse(result_handle)

    # Open a CSV file to write the results
    with open(results_csv, 'w', newline='') as csvfile:
        fieldnames = ['Query ID', 'Query Sequence', 'Subject', 'E-value', 'Score', 'Align Length', 'Identities', 'Match']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, quoting=csv.QUOTE_MINIMAL)

        # Write the header
        writer.writeheader()

        # Process BLAST results and write to CSV
        for blast_record in blast_records:
            query_id = blast_record.query.split()[0]  # Assumes the first word in the query description is the ID
            query_seq = query_sequences.get(query_id, "NA")  # Get the sequence or NA if not found
            for alignment in blast_record.alignments:
                for hsp in alignment.hsps:
                    if hsp.expect < e_value_threshold:
                        writer.writerow({
                            'Query ID': query_id,
                            'Query Sequence': query_seq,
                            'Subject': alignment.title.replace(',', ';'),  # Replace commas in subject title
                            'E-value': hsp.expect,
                            'Score': hsp.score,
                            'Align Length': hsp.align_length,
                            'Identities': hsp.identities,
                            'Match': hsp.match.replace(',', ';')  # Replace commas in matches
                        })


In [ ]:
import numpy as np
import sys
import torch
import csv
import shutil
from typing import List, Tuple, Union

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio.Blast import NCBIXML
from Bio.Blast import NCBIWWW
from Bio import Align
from Bio.PDB import PDBParser, PDBIO
from Bio.SeqUtils import seq1

import biotite.structure.io as pdb
import biotite.structure as struct
import numpy as np
import csv
import subprocess
import os

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig
import gc

from transformers import (
    AutoTokenizer,
    EsmForProteinFolding,
)
import biotite.structure.io as bsio

def read_prompts(input_file):
    atseqs = []
    with open(input_file, encoding='utf-8-sig', newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            atseqs.append(row[0])
    atseqs = np.array(atseqs)
    #seq_ids = np.array(seq_ids)
    #atseqs = [' '.join(inner_list) for inner_list in atseqs]
    return atseqs

def model_load(adapter_name):
    model_name = "togethercomputer/evo-1-8k-base"
    revision = "1.1_fix"
    config = AutoConfig.from_pretrained(model_name, trust_remote_code=True, revision=revision)
    model = AutoModelForCausalLM.from_pretrained(
            model_name,
            config=config,
            trust_remote_code=True, revision=revision).to("cuda:0")
    model.config.use_cache = True
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.add_special_tokens({'eos_token': ' '})
    tokenizer.pad_token = tokenizer.eos_token
    adapter = adapter_name
    model.load_adapter(adapter)
    model.config.use_cache = True
    model.eval()
    return model, tokenizer

def run_model(current_aca_seqs, model, tokenizer):
    print(current_aca_seqs)
    input_ids = tokenizer(current_aca_seqs, return_tensors="pt").to('cuda:0')
    del input_ids['token_type_ids']
    temp = 1.0
    outputs = model.generate(
        **input_ids,
        max_new_tokens=1000,
        temperature= temp,
        repetition_penalty=None,
        top_k=4,
        top_p=1,
        penalty_alpha=None,
        do_sample=temp is not None,
        eos_token_id=tokenizer.eos_token_id)
    try:
        genseq = tokenizer.decode(outputs[0], skip_special_tokens=True)
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError encountered: {e}")
        genseq = "ATATATATGCGCGCGCGC"
    return genseq

def read_evo_seqs(csv_file_path):
    """Reads a CSV file, extracts and cleans data from two columns, and separates sequences and prompts based on clean and non-clean conditions."""
    clean_data = []  # List for clean prompts and sequences
    non_clean_data = []  # List for non-clean prompts and sequences
    chars_to_remove = ['`', '!', '@']  # Characters to check in prompts and remove from sequences

    with open(csv_file_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if row:  # Check if row is not empty
                prompt = row[0]
                sequence = row[1] if len(row) > 1 else ''  # Handle missing second column

                # Remove specified characters from the sequence
                for char in chars_to_remove:
                    sequence = sequence.replace(char, '')

                # Check if the prompt is clean and does not contain the specified characters
                if not any(char in prompt for char in chars_to_remove):
                    clean_data.append((prompt, sequence))
                else:
                    non_clean_data.append((prompt, sequence))

    return clean_data, non_clean_data

def get_rc(sequences,truth):
    dna_seq = [Seq(curr_seq) for curr_seq in sequences]
    if truth:
        rev_dna_seq = [curr_seq.reverse_complement() for curr_seq in dna_seq]
        #return rev_dna_seq
        return rev_dna_seq+dna_seq
    else:
        return dna_seq

def make_fasta(rc_sequences, prompts, output_file):
    dna_seq_record = [SeqRecord(dna_seq, id=prompt, description=prompt) for (dna_seq, prompt) in zip(rc_sequences, prompts)]
    with open(output_file, "w") as output_handle:
        SeqIO.write(dna_seq_record, output_handle, "fasta")

def run_prodigal(input_file, output_file, output_orf_file):
    cmd = f'/home/cirrascale/miniconda3/envs/evo-design/bin/prodigal -i {input_file} -a {output_file} -d {output_orf_file} -p meta'
    subprocess.run(cmd, shell=True)

def filter_full_proteins(truth,input_file,output_file):
    with open(output_file, "w") as out_handle:
        for record in SeqIO.parse(input_file, "fasta"):
            if truth:
                if len(record.seq) < 500:
                    SeqIO.write(record, out_handle, "fasta")
            else:
                SeqIO.write(record, out_handle, "fasta")

def align_and_save_high_scores(test_sequences_file, reference_sequences_file, output_fasta_file):
    # Create an aligner object
    aligner = Align.PairwiseAligner()
    # Configure the aligner to the specific needs
    aligner.mode = 'global'  # For global alignment; use 'local' for local alignment
    aligner.match_score = 2  # Score for matched characters
    aligner.mismatch_score = -1  # Penalty for mismatched characters
    aligner.open_gap_score = -0.5  # Penalty for opening a gap
    aligner.extend_gap_score = -0.1  # Penalty for extending a gap

    # Load sequences from files
    test_sequences = list(SeqIO.parse(test_sequences_file, "fasta"))
    reference_sequences = list(SeqIO.parse(reference_sequences_file, "fasta"))

    # List to store sequences with high alignment scores
    high_score_sequences = []

    # Iterate over each test sequence
    for test_seq in test_sequences:
        # Variable to track the highest score relative to sequence length
        highest_identity = 0

        # Compare each test sequence against each reference sequence
        for ref_seq in reference_sequences:
            # Calculate the score for the current pair
            score = aligner.score(test_seq.seq, ref_seq.seq)

            # Calculate the maximum possible score for this pair (if all characters matched)
            max_score = min(len(test_seq.seq), len(ref_seq.seq)) * aligner.match_score

            # Calculate the identity percentage
            identity_percentage = (score / max_score) * 100

            # Update the highest identity found for this test sequence
            if identity_percentage > highest_identity:
                highest_identity = identity_percentage

        # Check if the highest identity percentage for this test sequence is above 25%
        if highest_identity >= 40:
            # Append this test sequence to the list
            high_score_sequences.append(test_seq)

    # Write the sequences that meet the criteria to an output FASTA file
    SeqIO.write(high_score_sequences, output_fasta_file, "fasta")

def fold_proteins(input_file,output_path,file_name):
    try:
        os.makedirs(output_path, exist_ok=True)
        print(f"Directory '{output_path}' created successfully or already exists.")
    except OSError as error:
        print(f"Failed to create directory '{output_path}'. Error: {error}")
    esmfold = EsmForProteinFolding.from_pretrained("facebook/esmfold_v1")
    esmfold = esmfold.to('cuda:0')
    esmfold.esm = esmfold.esm.half()
    esmfold_tokenizer = AutoTokenizer.from_pretrained("facebook/esmfold_v1")
    end_file = '.pdb'
    for i, protein_record in enumerate(SeqIO.parse(input_file, "fasta")):
        protein_seq = str(protein_record.seq)[:-1] # remove stop codon
        print('Protein sequence: ', protein_seq)
        with torch.inference_mode():
            esmfold_in = esmfold_tokenizer([protein_seq], return_tensors="pt", add_special_tokens=False)
            esmfold_out = esmfold(**esmfold_in.to('cuda:0'))
            esmfold_out_pdb = esmfold.output_to_pdb(esmfold_out)[0]
        curr_pro = str(i)
        file_curr = output_path + '/' + file_name + curr_pro + end_file
        print(file_curr)
        with open(file_curr, "w") as f:
            f.write(esmfold_out_pdb)

def score_pdb(input_file_name,input_path,output_file,output_fasta):
    end_file = '.pdb'
    nfiles = len([name for name in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, name))])
    files_seqs = []
    with open(output_file, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['PDB File', 'Average pLDDT Score'])
        for i in range(0,nfiles):
            curr_pro = str(i)
            file_name = input_file_name + curr_pro + end_file
            structure = pdb.load_structure(file_name, extra_fields=["b_factor"])
            if isinstance(structure, struct.AtomArrayStack):
                av_b_factors = [np.mean(model.b_factor) for model in structure]
                average_plddt = np.mean(av_b_factors)
            else:
                b_factors = structure.b_factor
                average_plddt = np.mean(b_factors)
            #writer.writerow([file_name, average_plddt]) --> writes all scores to seperate file
            if average_plddt > 0.5:
                parser = PDBParser()
                structure = parser.get_structure("test", file_name)
                chains = {chain.id:seq1(''.join(residue.resname for residue in chain)) for chain in structure.get_chains()}
                writer.writerow([file_name, average_plddt,chains])
                files_seqs.append([file_name,"".join(list(chains.values()))])
    seq_records = []
    print(files_seqs)
    for description, sequence in files_seqs:
        # Create a SeqRecord object from the sequence string
        seq_record = SeqRecord(Seq(sequence),
                               id=description,  # Use the description as the record ID
                               description='')  # Keep the description field empty if not needed
        seq_records.append(seq_record)

    # Write the list of SeqRecord objects to a FASTA file
    SeqIO.write(seq_records, output_fasta, "fasta")

In [ ]:
def main():
    sort_aca_seqs = read_prompts('/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/at_prompts.csv')
    coupledpromptseqscores = []
    model, tokenizer = model_load('lsmille/lora_evo_ta_all_layers_16')
    n_sample_per_prompt= 50
    n_batches = len(sort_aca_seqs)
    for j in range(0,n_sample_per_prompt):
        for i in range(0,n_batches):
            genseqs = run_model(sort_aca_seqs[i], model, tokenizer)
            genseqs = [str(genseqs)]
            #genscores = [str(score) for score in genscores]
            currentpromptseqscores = [[prompt,seq] for prompt, seq in zip([sort_aca_seqs[i]],genseqs)]
            print(currentpromptseqscores)
            coupledpromptseqscores = coupledpromptseqscores + currentpromptseqscores
    np.savetxt('/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/test_gen16.csv', coupledpromptseqscores, delimiter=',', fmt='%s')
    randprompseqsdata,atseqsdata = read_evo_seqs('/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/test_gen16.csv')
    sequences = [sequence for _, sequence in atseqsdata]
    prompts = [prompt for prompt, _ in atseqsdata]
    rc_sequences = get_rc(sequences,True)
    make_fasta(rc_sequences,prompts,'/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb_16_seqs.fasta')
    run_prodigal('/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb_16_seqs.fasta', '/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb_16_proteins.fasta','/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb_16_orfs.fasta')
    filter_full_proteins(True,'/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb_16_proteins.fasta','/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb_16_filt_proteins.fasta')
    align_and_save_high_scores('/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb_16_filt_proteins.fasta','/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/real_at_protein_filt_seqs.fasta','/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb_16_msa_filt_proteins.fasta')
    fold_proteins('/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb15_antitoxin_seqs.fasta','/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb15/pdb','lb15at_hmm_pdb')
    fold_proteins('/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb15_toxin_seqs.fasta','/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb15/pdb','lb15t_hmm_pdb')
    fold_proteins('/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb15_sequences_blast.fasta','/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb15/pdb','lb15blast_pdb')
    score_pdb(
        '/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb15/pdb/lb15at_hmm_pdb',
        '/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb15/pdb',
        '/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb15/pdb/lb15at_scored_pdb.csv',
        '/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb15/pdb/lb15at_pdb_filt.fasta')
    score_pdb(
        '/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb15/pdb/lb15t_hmm_pdb',
        '/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb15/pdb',
        '/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb15/pdb/lb15t_scored_pdb.csv',
        '/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb15/pdb/lb15t_pdb_filt.fasta')
    score_pdb(
        '/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb15/pdb/lb15blast_hmm_pdb',
        '/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb15/pdb',
        '/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb15/pdb/lb15blast_scored_pdb.csv',
        '/large_experiments/hielab/adititm/base_evo_tests/dna-gen/eval/toxin_antitoxin/lb15/pdb/lb15blast_pdb_filt.fasta')
main()

In [ ]:
if __name__ == "__main__":
    main()